# INSTRUCTIONS ON HOW TO RUN:

1.  Click on: "Open in playground" (top left, above this text box and under "File")

2.  Click on: "Connect" (top right, above this text box under "Share" and next to "Editing")

3.  Sign in if necessary

3.  Click on: "Runtime" (5th tab from the right from the Colab Logo (Orange Infinity Sign))

4.  And Then Click on: "Run all" (top first option)

5.  Wait... and then interact!

Navigate by clicking the side icon for the Table of contents

In [ ]:
%matplotlib inline

from ipywidgets import interact, interactive, FloatSlider, fixed
from IPython.display import clear_output, display, HTML

import math
# import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation
import matplotlib.gridspec as gridspec

plt.style.use('seaborn-darkgrid')
plt.rcParams.update({"axes.edgecolor" : "w"})

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Phase Spaces

In [ ]:
def plot_diff_eq(
    x_label, y_label, get_x_dot, get_y_dot,
    X_min, X_max, Y_min, Y_max,
    plot_type, DELTA_T, **kwargs):
  x_values, y_values = np.meshgrid(
      np.arange(X_min, X_max, DELTA_T), 
      np.arange(Y_min, Y_max, DELTA_T))
  x_dot, y_dot =  (get_x_dot(x_values, y_values, **kwargs), 
                   get_y_dot(x_values, y_values, **kwargs))
  
  xy = np.sqrt(x_dot**2, y_dot**2)
  fig = plt.figure(figsize=(16,8))
  if (plot_type == "Streamplot Density"):
    plt.streamplot(x_values, y_values, x_dot, y_dot)
  elif (plot_type == "Streamplot Line Width"):
    lw = 5 * xy / xy.max()
    plt.streamplot(x_values, y_values, x_dot, y_dot, 
                   color="k", linewidth=lw)
  elif (plot_type[:16] == "Streamplot Color"):
    color_dot = x_dot if plot_type[-2] == " x" else (
                  y_dot if plot_type[-2] == " y" else xy)
    strm = plt.streamplot(x_values, y_values, x_dot, y_dot, 
                          color=color_dot, linewidth=2, cmap='autumn')
    fig.colorbar(strm.lines)
  elif (plot_type == "Quiver Mid Default"):
    plt.quiver(x_values, y_values, x_dot, y_dot, units="xy", pivot="mid")
  else:
    plt.quiver(x_values, y_values, x_dot, y_dot, units="xy", pivot="tail", 
               width=0.015, scale= 1/0.012)
    plt.scatter(x_values, y_values, color="0.5", s=1)

  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.show()

def plot_diff_eq_interactive(
    x_label, y_label, get_x_dot, get_y_dot,
    X_min=(-30, -1), X_max=(1, 32), 
    Y_min=(-14, -1), Y_max=(1, 16), **ckwargs):
  e_d = lambda dimen: fixed(dimen) if type(dimen) is int else dimen
  return interactive(
    lambda **kwargs: 
      plot_diff_eq(
        x_label, y_label, 
        get_x_dot, get_y_dot, **kwargs),
      X_min=e_d(X_min), X_max=e_d(X_max), 
      Y_min=e_d(Y_min), Y_max=e_d(Y_max),
      plot_type=["Streamplot Density", "Streamplot Line Width", 
                  "Streamplot Color x", "Streamplot Color y", "Streamplot Color xy",
                  "Quiver Mid Default", "Quiver Tail Min"],
      DELTA_T=(.001, .3, .001), **ckwargs)

### Predator vs Prey model:
$$
\begin{aligned}
\dot{x} & = x (\alpha -\beta y) \\
\dot{y} & = y (\delta \beta x -\gamma)
\end{aligned}
$$

In [ ]:
def pred_prey_get_x_dot(prey, predator, alpha, beta, **kwargs):
  return prey * (alpha - beta * predator)

def pred_prey_get_y_dot(prey, predator, beta, delta, gamma, **kwargs):
  return predator * (delta * beta * prey - gamma)

w = plot_diff_eq_interactive("Prey", "Predator", 
      pred_prey_get_x_dot, pred_prey_get_y_dot, X_min=0, Y_min=0,
      alpha=(-10., 12), beta=(-10., 12), delta=(-10., 12), gamma=(-10., 12))
      #prey growth    #prey hunted    #prey num for predator #predator death
display(w)

interactive(children=(IntSlider(value=16, description='X_max', max=32, min=1), IntSlider(value=8, description=…

### Pendulum with air resistance:
$$
\begin{aligned}
\ddot{\theta}(t) = 
  -\mu \dot{\theta}(t)
  -\frac{g}{L} \sin(\theta(t)) 
\end{aligned}
$$

In [ ]:
def pendulum_get_x_dot(x_values, y_values, **kwargs):
  return y_values

def pendulum_get_y_dot(theta, theta_dot, g, L, mu):
  return -mu * theta_dot - (g / L) * np.sin(theta)

w = plot_diff_eq_interactive("Theta", "Theta Dot", 
      pendulum_get_x_dot, pendulum_get_y_dot,
      g=(0., 20), L=(.001, 10, .001), mu=(0., 1))
display(w)

interactive(children=(IntSlider(value=-16, description='X_min', max=-1, min=-30), IntSlider(value=16, descript…

### Hooke's Law with damping:
$$
\begin{aligned}
\frac{d^2x}{dt^2} =
  \frac{1}{m}(-kx -\gamma \frac{dx}{dt})
\end{aligned}
$$

In [ ]:
def spring_get_x_dot(x_values, y_values, **kwargs):
  return y_values

def spring_get_y_dot(pos, vel, k, m, gamma):
  return (1/m) * (-k * pos -gamma * vel)

w = plot_diff_eq_interactive("Position", "Velocity", 
      spring_get_x_dot, spring_get_y_dot,
      k=(0., 10), m=(0., 6), gamma=(0., 1))
display(w)

interactive(children=(IntSlider(value=-17, description='X_min', max=-1, min=-32), IntSlider(value=16, descript…

# Approximating First Order Differential Equations

### Newton's Law of Cooling:
$$
\begin{aligned}
\frac{dT}{dt} = -k(T - T_a)
\end{aligned}
$$

In [ ]:
LABELS = ("Temperature", "Change in Temperature", "Time")

def multi_approx_heat_diff_eq(x_order, y_order, 
    DELTA_TIME, TOT_TIME, DELTA_T, init_T_min, init_T_max, **kwargs):
  
  def temp_diff_eq(Temp, k, Temp_a):
    return -k * (Temp - Temp_a)

  fig = plt.figure(figsize=(16,8))
  init_Temp_list = np.arange(init_T_min, init_T_max, DELTA_T)
  for T_0 in init_Temp_list:
    x_list, y_list = [], []
    for t in np.arange(0, TOT_TIME, DELTA_TIME):
      T_1 = temp_diff_eq(T_0, **kwargs)
      T_0 += T_1 * DELTA_TIME
      values = (T_0, T_1, t)
      x_list.append(values[x_order])
      y_list.append(values[y_order])
    plt.plot(x_list, y_list)
  plt.xlabel(LABELS[x_order])
  plt.ylabel(LABELS[y_order])
  plt.show()

w = interactive(multi_approx_heat_diff_eq,
      x_order=[(LABELS[-1], -1), (LABELS[0], 0)], 
      y_order=[(LABELS[0], 0), (LABELS[1], 1)],
      DELTA_TIME=(.001, .3, .001), TOT_TIME=(1, 19), 
      DELTA_T=(.1, 9.9), init_T_min=(-100,1), init_T_max=(1,100),
      k=(0., 1), Temp_a=(-100, 100))
display(w)

interactive(children=(Dropdown(description='x_order', options=(('Time', -1), ('Temperature', 0)), value=-1), D…

### Predator vs Prey model:
(System of 2 Differential Equations)
$$
\begin{aligned}
\frac{dx}{dt} & = x (\alpha -\beta y) \\
\frac{dy}{dt} & = y (\delta \beta x -\gamma)
\end{aligned}
$$

In [ ]:
def pred_prey_get_x_dot(prey, predator, alpha, beta, **kwargs):
  return prey * (alpha - beta * predator)

def pred_prey_get_y_dot(prey, predator, beta, delta, gamma, **kwargs):
  return predator * (delta * beta * prey - gamma)

w = plot_diff_eq_interactive("Prey", "Predator", 
      pred_prey_get_x_dot, pred_prey_get_y_dot, X_min=0, Y_min=0,
      alpha=(-10., 12), beta=(-10., 12), delta=(-10., 12), gamma=(-10., 12))
      #prey growth    #prey hunted    #prey num for predator #predator death
display(w)

interactive(children=(IntSlider(value=16, description='X_max', max=32, min=1), IntSlider(value=8, description=…

In [ ]:
LABELS = ("Number of Prey", "Change in Number of Prey", 
          "Number of Predators", "Change in Number of Predators",
          "Time")

def multi_approx_pred_prey_first_order_diff_eqs(x_order, y_order, 
    DELTA_TIME, TOT_TIME, DELTA_Prey, init_Prey_max, init_Pred, **kwargs):
  
  def prey_diff_eq(prey, predator, alpha, beta, **kwargs):
    return prey * (alpha - beta * predator)

  def pred_diff_eq(prey, predator, beta, delta, gamma, **kwargs):
    return predator * (delta * beta * prey - gamma)

  fig = plt.figure(figsize=(16,8))  
  init_Prey_list = np.arange(0, init_Prey_max, DELTA_Prey)
  for prey_0 in init_Prey_list:
    pred_0 = init_Pred
    x_list, y_list = [], []
    for t in np.arange(0, TOT_TIME, DELTA_TIME):
      prey_1 = prey_diff_eq(prey_0, pred_0, **kwargs)
      pred_1 = pred_diff_eq(prey_0, pred_0, **kwargs)
      prey_0 += prey_1 * DELTA_TIME
      pred_0 += pred_1 * DELTA_TIME
      values = (prey_0, prey_1, pred_0, pred_1, t)
      x_list.append(values[x_order])
      y_list.append(values[y_order])
    plt.plot(x_list, y_list)
  plt.xlabel(LABELS[x_order])
  plt.ylabel(LABELS[y_order])
  plt.show()

w = interactive(multi_approx_pred_prey_first_order_diff_eqs,
      x_order=[(LABELS[-1], -1)] + [(v, i) for i, v in enumerate(LABELS[:-2])], 
      y_order=[(v, i) for i, v in enumerate(LABELS[:-1])],
      DELTA_TIME=(.001, .4, .001), TOT_TIME=(1, 499), 
      DELTA_Prey=(2, 20), init_Prey_max=(0, 100), init_Pred=(0, 20),
      alpha=(-10., 12), beta=(-10., 12), delta=(-10., 12), gamma=(-10., 12))
display(w)

interactive(children=(Dropdown(description='x_order', options=(('Time', -1), ('Number of Prey', 0), ('Change i…

### SIR Model:
(System of 3 Differential Equations)
$$
\begin{aligned}
\frac{dS}{dt} & = -\frac{\beta SI}{N} \\
\frac{dI}{dt} & = 
  \frac{\beta SI}{N} -\gamma I \\
\frac{dR}{dt} & = \gamma I
\end{aligned}
$$

In [ ]:
def sir_model(plot_type, DELTA_TIME=.1, TOT_TIME=60,
    init_I_percent=.01, N=1,
    beta=.6, #Infection rate
    gamma=.1): #Recovery rate            
  S = N * (1 - init_I_percent)
  I = N * init_I_percent
  R = 0
  t_list, S_list, I_list, R_list = ([] for i in range(4))

  for t in np.arange(0, TOT_TIME, DELTA_TIME):
    t_list.append(t)
    S_list.append(S)
    I_list.append(I)
    R_list.append(R)
    dS_dt = -beta * S * I/N 
    dI_dt = beta * S * I/N - gamma * I
    dR_dt = gamma * I
    S += dS_dt * DELTA_TIME
    I += dI_dt * DELTA_TIME
    R += dR_dt * DELTA_TIME

  print("Maximum Infectious population at a time :{}% \n".format(round(np.array(I_list).max()*100/N, 2)))

  plt.figure(figsize=(16,8))
  if (plot_type == "Lineplots"):
    plt.plot(t_list, S_list, label="Susceptible")
    plt.plot(t_list, I_list, label="Infected")
    plt.plot(t_list, R_list, label="Recovered")
  else:
    labels = ["Susceptible", "Infected", "Recovered"]
    y = np.vstack([S_list, I_list, R_list])
    plt.stackplot(t_list, y, labels=labels)

  plt.xlabel("Time")
  plt.ylabel("Population " + ("Count" if N > 1 else "Percentage"))
  plt.legend()
  plt.show()

w = interactive(sir_model, 
      plot_type=["Lineplots", "Stackplot"],
      DELTA_TIME=(.01, 1, .01), TOT_TIME=(0,100),  
      init_I_percent=(0.0, 1.0), N=(1, 1000), beta=(0.0, 1.0), gamma=(0.0, 1.0))
display(w)

interactive(children=(Dropdown(description='plot_type', options=('Lineplots', 'Stackplot'), value='Lineplots')…

# Approximating Second Order Differential Equations

In [ ]:
def approx_second_order_diff_eq(
    LABELS, get_second_order, 
    x_order, y_order, DELTA_T, TOT_T,
    initial_0, initial_1, **kwargs):
  x_0, x_1 = initial_0, initial_1 
  x_list, y_list = [], []

  for t in np.arange(0, TOT_T, DELTA_T):
    x_2 = get_second_order(x_0, x_1, **kwargs) #+ i_2
    x_0 += x_1 * DELTA_T
    x_1 += x_2 * DELTA_T
    values = (x_0, x_1, x_2, t)
    x_list.append(values[x_order])
    y_list.append(values[y_order])

  fig = plt.figure(figsize=(16,8))
  plt.plot(x_list, y_list)
  plt.xlabel(LABELS[x_order])
  plt.ylabel(LABELS[y_order])

  plt.title("DELTA_T={} \n\n{}".format(DELTA_T, 
    "  ".join([k+"="+str(v) for k, v in kwargs.items()])))

  plt.show()

def approx_second_order_diff_eq_interactive(
    label_0, label_1, label_2,
    get_second_order, i_0, i_1, 
    D_T=FloatSlider(min=.001, max=.5, step=.001, value=.1), 
    **ckwargs):
  LABELS = (label_0, label_1, label_2, "Time")
  return interactive(
    lambda **kwargs: 
      approx_second_order_diff_eq(
        LABELS, get_second_order, **kwargs),
      x_order=[(LABELS[-1], -1)] + [(v, i) for i, v in enumerate(LABELS[:-2])], 
      y_order=[(v, i) for i, v in enumerate(LABELS[:-1])],
      DELTA_T=D_T, TOT_T=(1, 999),
      initial_0=i_0, initial_1=i_1, **ckwargs)

### Pendulum with air resistance:
$$
\begin{aligned}
\ddot{\theta}(t) = 
  -\mu \dot{\theta}(t)
  -\frac{g}{L} \sin(\theta(t)) 
\end{aligned}
$$

In [ ]:
def pendulum_diff_eq(theta, theta_dot, g, L, mu):
  return -mu * theta_dot - (g / L) * np.sin(theta)

w = approx_second_order_diff_eq_interactive(
      "Theta", "Theta Dot", "Theta Double Dot", 
      pendulum_diff_eq, i_0=(-np.pi, 2*np.pi, .01), i_1=(-10., 10),
      g=(0., 20), L=(.001, 10, .001), mu=(0., 1))
display(w)

interactive(children=(Dropdown(description='x_order', options=(('Time', -1), ('Theta', 0), ('Theta Dot', 1)), …

### Hooke's Law with damping:
$$
\begin{aligned}
\frac{d^2x}{dt^2} =
  \frac{1}{m}(-kx -\gamma \frac{dx}{dt})
\end{aligned}
$$

In [ ]:
# TIME VS POSITION????
# NOT LOW ENOUGH DELTA_T WITH GAMMA SET TO .4

def spring_diff_eq(pos, vel, k, m, gamma):
  return (1/m) * (-(k * pos) -(gamma * vel))

w = approx_second_order_diff_eq_interactive(
      "Position", "Velocity", "Acceleration", 
      spring_diff_eq, i_0=(-4., 6), i_1=(-10., 10),
      k=(0., 10), m=(0., 6), gamma=(0., 2, .01))
display(w)

interactive(children=(Dropdown(description='x_order', options=(('Time', -1), ('Position', 0), ('Velocity', 1))…

In [ ]:
# VERY LOW DELTA_T WITH GAMMA SET TO 0

w = approx_second_order_diff_eq_interactive(
      "Position", "Velocity", "Acceleration", 
      spring_diff_eq, i_0=(-4., 6), i_1=(-10., 10),
      # k=(0., 10), m=(0., 6), gamma=(0., 1))
      D_T=fixed(.00001),
      k=(0., 10), m=(0., 6), gamma=fixed(0))
display(w)

interactive(children=(Dropdown(description='x_order', options=(('Time', -1), ('Position', 0), ('Velocity', 1))…

In [ ]:
# VERY LOW DELTA_T WITH GAMMA SET TO >0

w = approx_second_order_diff_eq_interactive(
      "Position", "Velocity", "Acceleration", 
      spring_diff_eq, i_0=(-4., 6), i_1=(-10., 10),
      # k=(0., 10), m=(0., 6), gamma=(0., 1))
      D_T=fixed(.00001),
      k=(0., 10), m=(0., 6), gamma=fixed(.5))
display(w)

interactive(children=(Dropdown(description='x_order', options=(('Time', -1), ('Position', 0), ('Velocity', 1))…

### Drag Force:
$$
\begin{aligned}
\left\lvert{
    \frac{d^2x}{dt^2}
  }\right\rvert =
  \frac{1}{m} (\frac{1}{2} C \rho A (\frac{dx}{dt})^2)
\end{aligned}
$$

In [ ]:
# TIME VS POSITION????
# NOT LOW ENOUGH DELTA_T WITH INITIAL VELOCITY SET TO 3

def drag_force_diff_eq(pos, vel, m, coeff, rho, area):
  return (math.copysign(1, -vel) * 
    (1/m) * (1/2) * coeff * rho * area * vel ** 2)

w = approx_second_order_diff_eq_interactive(
      "Position", "Velocity", "Acceleration", 
      drag_force_diff_eq, i_0=(-10, 10), i_1=(-10, 10),
      m=(0., 6), coeff=(0., 2), rho=(0., 10), area=(0., 10))
display(w)

interactive(children=(Dropdown(description='x_order', options=(('Time', -1), ('Position', 0), ('Velocity', 1))…

In [ ]:
# VERY LOW DELTA_T WITH INITIAL VELOCITY SET TO 3

w = approx_second_order_diff_eq_interactive(
      "Position", "Velocity", "Acceleration", 
      drag_force_diff_eq, i_0=(-10, 10), i_1=fixed(3),
      D_T=fixed(.0001),
      m=(0., 6), coeff=(0., 2), rho=(0., 10), area=(0., 10))
display(w)

interactive(children=(Dropdown(description='x_order', options=(('Time', -1), ('Position', 0), ('Velocity', 1))…

### Bungee Jumping (Where down is positive)
$$
\begin{aligned}
\frac{d^2x}{dt^2} =
  \frac{1}{m}(mg -kx -\gamma \frac{dx}{dt})
\end{aligned}
$$

In [10]:
#@title
def bungee_diff_eq(pos, vel, g, k, m, gamma):
  return (1/m) * ((m * g) -(k * pos) -(gamma * vel))

def approx_second_order_diff_eq_interact(
    label_0, label_1, label_2,
    get_second_order, i_0, i_1, 
    D_T=FloatSlider(min=.001, max=.5, step=.001, value=.1), 
    **ckwargs):
  LABELS = (label_0, label_1, label_2, "Time")
  return interact(
    lambda **kwargs: 
      approx_second_order_diff_eq(
        LABELS, get_second_order, **kwargs),
      x_order=[(LABELS[-1], -1)] + [(v, i) for i, v in enumerate(LABELS[:-2])], 
      y_order=[(v, i) for i, v in enumerate(LABELS[:-1])],
      DELTA_T=D_T, TOT_T=(1, 999),
      initial_0=i_0, initial_1=i_1, **ckwargs)

approx_second_order_diff_eq_interact(
      "Position", "Velocity", "Acceleration",
      bungee_diff_eq, i_0=(-100., 100), i_1=(-10., 10),
      g=(-10., 20), k=(0., 4), m=(0., 120), gamma=(0., 12, .01))

interactive(children=(Dropdown(description='x_order', options=(('Time', -1), ('Position', 0), ('Velocity', 1))…

<function __main__.approx_second_order_diff_eq_interact.<locals>.<lambda>>

# Using Scipy to Solve Differential Equations

In [ ]:
from scipy import integrate

### Lorenz system:
$$
\begin{aligned}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x -y -xz \\
\dot{z} & = -\beta z +xy
\end{aligned}
$$

In [ ]:
def solve_lorenz(MAX_TIME=20, N=20, 
    angle=140, sigma=10.0, beta=8.0/3, rho=28.0):
  
  def lorenz_diff_eqs(x_y_z, t0):
    x, y, z = x_y_z
    return [  sigma * (y - x), 
              x * (rho - z) - y, 
              x * y - beta * z]

  t = np.linspace(0, MAX_TIME, int(250*MAX_TIME))
  
  np.random.seed(1)
  init_X_arr = -15 + 30 * np.random.random((N, 3))

  X = np.asarray([
          integrate.odeint(lorenz_diff_eqs, init_X, t)
            for init_X in init_X_arr])
  
  fig = plt.figure(figsize=(16,8))
  ax = fig.add_axes([0, 0, 1, 1], projection='3d')
  ax.set_xlim((-25, 25))
  ax.set_ylim((-35, 35))
  ax.set_zlim((5, 55))
  colors = plt.cm.jet(np.linspace(0, 1, N))
  for i in range(N):
    x, y, z = X[i,:,:].T
    lines = ax.plot(x, y, z, '-', c=colors[i])
    plt.setp(lines, linewidth=2)
  ax.view_init(30, angle)
  plt.show()

w = interactive(solve_lorenz, MAX_TIME=(0, 50), N=(0, 50), 
      angle=(0.0, 360.0), sigma=(0.0, 50.0), rho=(0.0, 50.0))
display(w)

interactive(children=(IntSlider(value=20, description='MAX_TIME', max=50), IntSlider(value=20, description='N'…

### Predator vs Prey model:
$$
\begin{aligned}
\dot{x} & = x (\alpha -\beta y) \\
\dot{y} & = y (\delta \beta x -\gamma)
\end{aligned}
$$

In [ ]:
def solve_pred_prey(plot_type, 
    MAX_TIME=20, N=10, init_Prey=20, init_Pred=5,
    alpha=2.0, #prey growth
    beta=.1, #prey hunted
    delta=3, #prey num for predator
    gamma=.75): #predator death

  def pred_prey_diff_eqs(x_y, t=0):
    prey, predator = x_y
    return [  prey * (alpha - beta * predator),
              predator * (delta * beta * prey - gamma)]

  t = np.linspace(0, MAX_TIME, int(250*MAX_TIME))
  pop_equilib = np.array([gamma/(delta*beta), alpha/beta])
  print("Population Equilbirium at X=({:.2f}, {:.2f}) \n".format(*pop_equilib))

  fig = plt.figure(figsize=(16,8))
  if (plot_type == "Population vs Time"):
    init_X = np.array([init_Prey, init_Pred])
    X = integrate.odeint(pred_prey_diff_eqs, init_X, t)
    prey, predator = X.T
    plt.plot(t, prey, "b-", label="Prey")
    plt.plot(t, predator, "r-", label="Predator")
    plt.xlabel("Time")
    plt.ylabel("Population")        
  else:
    init_val = np.linspace(0.3, 0.9, N)                         
    colors = plt.cm.autumn_r(np.linspace(0.3, 1., len(init_val)))
    for v, col in zip(init_val, colors):
      init_X = v * pop_equilib                            
      X = integrate.odeint(pred_prey_diff_eqs, init_X, t)   
      plt.plot(*X.T, lw=3.5*v, color=col, 
        label="X_i=({:.2f}, {:.2f})".format(init_X[0], init_X[1])) 
    plt.plot(*pop_equilib, 'ro') 
    plt.xlabel("Prey")
    plt.ylabel("Predator")

  plt.legend()
  plt.show()

w = interactive(solve_pred_prey, plot_type=["Population vs Time", "Predator vs Prey"],
      MAX_TIME=(0, 50), N=(0, 20), init_Prey=(0, 50), init_Pred=(0, 50),
      angle=(0.0, 5.0), sigma=(0.0, 5.0), rho=(0.0, 5.0))
display(w)

interactive(children=(Dropdown(description='plot_type', options=('Population vs Time', 'Predator vs Prey'), va…

### Zombie Apocalypse:
(Variation of the SIR Model with vital dynamics)
$$
\begin{aligned}
\frac{dS}{dt} & = \Pi -\beta SZ -\delta S \\
\frac{dZ}{dt} & = \beta SZ +\zeta R -\alpha SZ \\
\frac{dR}{dt} & = \delta S +\alpha SZ -\zeta R
\end{aligned}
$$

In [ ]:
def solve_zombie_apoc(plot_data, plot_type, MAX_TIME=50, 
    init_S=100, init_Z=0, init_R=0,
    Pi=10, # Birth rate of susceptibles
    delta=.0001, # Death rate of susceptibles
    beta=.01, # Rate a susceptible becomes a zombie
    zeta=.0025, # Rate a dead person becomes a zombie
    alpha=.001): # Rate a zombie becomes destroyed

  def zombie_apoc_diff_eqs(S_Z_R, t=0):
    S, Z, R = S_Z_R
    return [
      Pi - beta * S * Z - delta * S,
      beta * S * Z + zeta * R - alpha * S * Z,
      delta * S + alpha * S * Z - zeta * R]
  
  t = np.linspace(0, MAX_TIME, int(250*MAX_TIME))
  init_X = np.array([init_S, init_Z, init_Z])
  X = integrate.odeint(zombie_apoc_diff_eqs, init_X, t)
  S, Z, R = X.T

  fig = plt.figure(figsize=(16,8))
  if (plot_type == "Lineplots"):
    plt.plot(t, S, label="Living", color="tab:blue")
    plt.plot(t, Z, label="Zombies", color="tab:green")
    if (plot_data != "Living vs Zombies"):
      plt.plot(t, R, label="Decesased", color="tab:grey")
  else:
    stack = [S, Z]
    labels = ["Living", "Zombies"]
    colors = ["tab:blue", "tab:green"]
    if (plot_data != "Living vs Zombies"):
      stack.append(R)
      labels.append("Dceased")
      colors.append("tab:grey")
    y = np.vstack(stack)
    plt.stackplot(t, y, labels=labels, colors=colors)

  plt.xlabel("Time")
  plt.ylabel("Population Count")
  plt.legend(loc=2)
  plt.show()

w = interactive(solve_zombie_apoc, 
      plot_data=["Living vs Zombies", "Living vs Zombies vs Deceased"],
      plot_type=["Lineplots", "Stackplot"],
      MAX_TIME=(0, 1000), init_S=(0, 1000), init_Z=(0, 1000), init_R=(0, 1000),
      Pi=(0, 1, .0001), delta=(0, 1, .0001), 
      beta=(0, 1, .0001), zeta=(0, 1, .0001), alpha=(0, 1, .0001))
display(w)

interactive(children=(Dropdown(description='plot_data', options=('Living vs Zombies', 'Living vs Zombies vs De…

### Heat equation:
$$
\begin{aligned}
\frac{\partial T}{\partial t} = 
  \alpha (
    \frac{\partial^2 T}{\partial x^2} +
    \frac{\partial^2 T}{\partial y^2} +
    \frac{\partial^2 T}{\partial z^2}
  ) = \alpha \nabla^2 T
\end{aligned}
$$